To accentuate the white matter and gray matter in the brain proton density image, we need to apply two intensity transformations:

White Matter: Typically has a higher intensity range in MRI images.
Gray Matter: Typically has a mid-range intensity.

steps needed:

Step 1: Load the brain image.
Step 2: Apply an intensity transformation to highlight white matter and gray matter.
Step 3: Show intensity transformation plots.